In [41]:
import requests
import pandas as pd
import defs
from datetime import datetime, timedelta
import pytz
import defs

Debo obtener la data de ESTA semana. Parametros en RFC3339.  
Considerando que solo obtengo 500 velas por llamada de la API y que cada dia tengo 1440 velasde M1, tengo que hacer 3 llamadas de la API por cada dia de la semana.
- Inicio: 2024-06-24T04:00:00.000000000Z  #hora de NY UTC-4 para esta semana
- Inicio: 2024-06-24T04:00:00.000000000Z  
  
El API nos regresa de forma nativa la hora UTC-0 con la Z, por lo que la convertiremos a UTC-4 (hora actual de NY).  
Notar que en RFC339, la variable Z se usa para indicar que es UTC+0, por lo que la usare siempre (https://medium.easyread.co/understanding-about-rfc-3339-for-datetime-formatting-in-software-engineering-940aa5d5f68a).  


In [30]:
#Funcion para obtener hora actual de NY en formato Z UTC-0
import pytz
from datetime import datetime

def obtener_hora_nueva_york_rfc3339():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Formatear la fecha y hora en el formato deseado
    hora_formateada = ahora_ny.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_formateada

# Ejemplo de uso
hora_actual_ny = obtener_hora_nueva_york_rfc3339()
print("Hora actual en Nueva York:", hora_actual_ny)


Hora actual en Nueva York: 2024-06-26T15:03:33.486168Z


Voy a pedile a a API que me entregue las velas los ultimos 7 dias siguiendo la hora de ny que es -4 . Yo manualmente le pongo la hora de inicio para simplificar este paso, luego tendra que ser automatizado.

In [66]:
now_time = obtener_hora_nueva_york_rfc3339()

def restar_siete_dias(fecha_hora_rfc3339):
    # Convertir la cadena RFC3339 a un objeto datetime
    fecha_hora = datetime.strptime(fecha_hora_rfc3339, "%Y-%m-%dT%H:%M:%S.%fZ")
    
    # Restar 7 días a la fecha y hora
    fecha_hora_menos_siete_dias = fecha_hora - timedelta(days=7)
    
    # Formatear la nueva fecha y hora en formato RFC3339 con la 'Z'
    nueva_fecha_hora_rfc3339 = fecha_hora_menos_siete_dias.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
    return nueva_fecha_hora_rfc3339

restar_siete_dias(now_time)


'2024-06-21T04:11:50.257508Z'

In [67]:
# Obtener las 04:00 hras de la hora del dia hace 7 dias

def obtener_hora_04_00(fecha_hora_rfc3339):
    # Convertir la cadena RFC3339 a un objeto datetime
    fecha_hora = datetime.strptime(fecha_hora_rfc3339, "%Y-%m-%dT%H:%M:%S.%fZ")
    
    # Crear un nuevo objeto datetime con la hora fijada a las 04:00 del mismo día
    fecha_04_00 = fecha_hora.replace(hour=4, minute=0, second=0, microsecond=0)
    
    # Formatear la nueva fecha y hora en formato RFC3339 con la 'Z'
    nueva_fecha_hora_rfc3339 = fecha_04_00.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
    return nueva_fecha_hora_rfc3339

# Ejemplo de uso
fecha_hora_original = "2024-06-21T01:28:14.265493Z"
fecha_hora_04_00 = obtener_hora_04_00(fecha_hora_original)
print("Fecha y hora original:", fecha_hora_original)
print("Fecha y hora a las 04:00:", fecha_hora_04_00)



Fecha y hora original: 2024-06-21T01:28:14.265493Z
Fecha y hora a las 04:00: 2024-06-21T04:00:00.000000Z


In [73]:
# Intento de obtener la fecha de inicio

hora_actual_ny = obtener_hora_nueva_york_rfc3339()
hace_7_dias = restar_siete_dias(hora_actual_ny)
hace_7_dias_4hrs = obtener_hora_04_00(hace_7_dias)
hace_7_dias_4hrs #esta es mi hora de inicio

'2024-06-21T04:00:00.000000Z'

Llamada al APi

In [91]:
session = requests.Session()
instrument = "EUR_USD"
granularity = "M1"
starting_date = hace_7_dias_4hrs
url = f"{defs.OANDA_URL}/instruments/{instrument}/candles"
params = {
    'granularity' : granularity,
    'price' : "MBA",
    'from' : starting_date,
    'count': 5000
}

response = session.get(url, params=params, headers=defs.SECURE_HEADER)
data = response.json()
data.keys()

dict_keys(['instrument', 'granularity', 'candles'])

In [75]:
len(data['candles'])

5000

In [76]:
prices = ["mid", "bid", "ask"]
ohlc = ["o", "h", "l", "c"]

In [77]:
our_data = []

for candle in data["candles"]:
    if candle["complete"] == False:
        continue
    new_dict = {}
    new_dict["time"] = candle["time"]
    new_dict["volume"] = candle["volume"]
    for price in prices:
        for oh in ohlc:
            new_dict[f"{price}_{oh}"] = candle[price][oh]
    our_data.append(new_dict)

print (our_data[0])

{'time': '2024-06-21T04:00:00.000000000Z', 'volume': 28, 'mid_o': '1.07162', 'mid_h': '1.07164', 'mid_l': '1.07156', 'mid_c': '1.07159', 'bid_o': '1.07155', 'bid_h': '1.07156', 'bid_l': '1.07149', 'bid_c': '1.07152', 'ask_o': '1.07170', 'ask_h': '1.07172', 'ask_l': '1.07164', 'ask_c': '1.07166'}


In [78]:
candles_df = pd.DataFrame.from_dict(our_data)
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-06-21T04:00:00.000000000Z,28,1.07162,1.07164,1.07156,1.07159,1.07155,1.07156,1.07149,1.07152,1.07170,1.07172,1.07164,1.07166
1,2024-06-21T04:01:00.000000000Z,12,1.07158,1.07162,1.07156,1.07160,1.07151,1.07155,1.07149,1.07152,1.07166,1.07169,1.07164,1.07167
2,2024-06-21T04:02:00.000000000Z,12,1.07158,1.07158,1.07153,1.07153,1.07150,1.07150,1.07145,1.07145,1.07166,1.07166,1.07160,1.07161
3,2024-06-21T04:03:00.000000000Z,20,1.07154,1.07156,1.07154,1.07155,1.07147,1.07149,1.07147,1.07148,1.07162,1.07163,1.07162,1.07162
4,2024-06-21T04:04:00.000000000Z,15,1.07153,1.07160,1.07153,1.07160,1.07146,1.07153,1.07146,1.07153,1.07160,1.07168,1.07160,1.07167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2024-06-26T16:19:00.000000000Z,58,1.06857,1.06866,1.06857,1.06866,1.06850,1.06859,1.06850,1.06859,1.06864,1.06873,1.06864,1.06873
4996,2024-06-26T16:20:00.000000000Z,50,1.06867,1.06874,1.06867,1.06868,1.06860,1.06867,1.06860,1.06860,1.06874,1.06881,1.06874,1.06875
4997,2024-06-26T16:21:00.000000000Z,33,1.06870,1.06871,1.06866,1.06868,1.06862,1.06864,1.06858,1.06861,1.06877,1.06878,1.06873,1.06876
4998,2024-06-26T16:22:00.000000000Z,69,1.06868,1.06874,1.06860,1.06861,1.06860,1.06867,1.06853,1.06854,1.06876,1.06881,1.06868,1.06868


In [90]:
a = candles_df.tail(1)['time']

4999    2024-06-26T16:23:00.000000000Z
Name: time, dtype: object


In [40]:
#Como eliminar el duplicado, hacer luego de juntar los dataframes
import pandas as pd

# Crear un DataFrame de ejemplo
data = {'Nombre': ['Ana', 'Luis', 'Ana', 'Carlos', 'Luis'],
        'Edad': [23, 34, 23, 45, 34],
        'Ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Barcelona']}

df = pd.DataFrame(data)

# Mostrar el DataFrame original
print("DataFrame original:")
print(df)

# Eliminar filas duplicadas
df_sin_duplicados = df.drop_duplicates()

# Mostrar el DataFrame sin duplicados
print("\nDataFrame sin duplicados:")
print(df_sin_duplicados)

DataFrame original:
   Nombre  Edad     Ciudad
0     Ana    23     Madrid
1    Luis    34  Barcelona
2     Ana    23     Madrid
3  Carlos    45   Valencia
4    Luis    34  Barcelona

DataFrame sin duplicados:
   Nombre  Edad     Ciudad
0     Ana    23     Madrid
1    Luis    34  Barcelona
3  Carlos    45   Valencia
